In [2]:
import galprime as gp

from astropy.io import fits
from astropy.table import Table

import os

from tqdm import tqdm

In [3]:

gp.combine_outputs("gprime_sims/gprime_out_bgsub/", "gprime_sims/gprime_out_bgsub_2/", 
                12261553, 12261630,  "gprime_bgsub/")

Combining profiles in bgsub_profiles: 100%|██████████| 25/25 [00:00<00:00, 733269.93it/s]
